In [1]:
import pdfplumber
import re
import pytesseract
from PIL import Image

def tokenize_pdf(pdf_path):
    import re
    import pdfplumber
    import pytesseract
    from PIL import Image

    TOK_RE = r"\w+|[^\w\s]"
    tokens = []      # for text‐only tokens
    table_tokens = []# for table tokens

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:

            # --- OCR fallback for embedded images ---
            if page.images:
                # render page at 300dpi to a PIL image
                page_img = page.to_image(resolution=300)
                pil_img  = page_img.original
                # run Tesseract
                ocr_text = pytesseract.image_to_string(pil_img)
                tokens += re.findall(TOK_RE, ocr_text, re.UNICODE)

            # 1) extract & tokenize tables
            tables = page.find_tables()
            for table in tables:
                for row in table.extract():
                    for cell in row:
                        if cell:
                            table_tokens += re.findall(TOK_RE, cell, re.UNICODE)

            # 2) extract text from the page and tokenize
            table_bboxes = [t.bbox for t in tables]
            words = page.extract_words()
            for word in words:
                word_bbox = (word['x0'], word['top'], word['x1'], word['bottom'])
                in_table = False
                for tb in table_bboxes:
                    if not (word_bbox[2] < tb[0] or word_bbox[0] > tb[2] or
                            word_bbox[3] < tb[1] or word_bbox[1] > tb[3]):
                        in_table = True
                        break
                if not in_table:
                    tokens += re.findall(TOK_RE, word['text'], re.UNICODE)

    return tokens, table_tokens


if __name__ == "__main__":
    pdf_path = r"C:\Users\ADMIN\Downloads\img20250420_19033735.pdf"
    toks, table_toks = tokenize_pdf(pdf_path)
    # show first 200 tokens for brevity
    print("Text tokens:")
    print(toks)
    print(f"... total text tokens: {len(toks)}")
    
    print("\nTable tokens:")
    print(table_toks)
    print(f"... total table tokens: {len(table_toks)}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [ ]:
# import pdfplumber
# import re

# def tokenize_pdf(pdf_path):
#     import re
#     import pdfplumber

#     TOK_RE = r"\w+|[^\w\s]"
#     tokens = []      # for text-only tokens
#     table_tokens = [] # for table tokens

#     with pdfplumber.open(pdf_path) as pdf:
#         for page in pdf.pages:
#             # 1) extract & tokenize tables
#             tables = page.find_tables()
#             for table in tables:
#                 for row in table.extract():             # row is a list of cell‑strings
#                     for cell in row:
#                         if cell:
#                             table_tokens += re.findall(TOK_RE, cell, re.UNICODE)

#             # 2) extract text from the page and tokenize
#             # Create a list of table bounding boxes to check against
#             table_bboxes = [table.bbox for table in tables]
            
#             # Extract all words from the page with their bounding boxes
#             words = page.extract_words()
#             for word in words:
#                 # Check if this word's bbox overlaps with any table bbox
#                 word_bbox = (word['x0'], word['top'], word['x1'], word['bottom'])
#                 in_table = False
#                 for table_bbox in table_bboxes:
#                     # Simple overlap check: not (word is completely left, right, above or below table)
#                     if not (word_bbox[2] < table_bbox[0] or  # word is left of table
#                             word_bbox[0] > table_bbox[2] or  # word is right of table
#                             word_bbox[3] < table_bbox[1] or  # word is above table
#                             word_bbox[1] > table_bbox[3]):   # word is below table
#                         in_table = True
#                         break
                
#                 # Only add tokens from words not in tables
#                 if not in_table:
#                     tokens += re.findall(TOK_RE, word['text'], re.UNICODE)

#     return tokens, table_tokens

# if __name__ == "__main__":
#     pdf_path = r"C:\Work\DOST-ITDI\Digital-Calibration-Certificate-for-National-Metrology-Laboratory-of-the-Philippines\valid pdf\3.pdf"
#     toks, table_toks = tokenize_pdf(pdf_path)
#     # show first 200 tokens for brevity
#     print("Text tokens:")
#     print(toks)
#     print(f"... total text tokens: {len(toks)}")
    
#     print("\nTable tokens:")
#     print(table_toks)
#     print(f"... total table tokens: {len(table_toks)}")

Text tokens:
['CALIBRATION', 'CERTIFICATE', 'No', '.', '02', '-', '2025', '-', 'FORC', '-', '0040', 'Date', 'of', 'Calibration', ':', 'February', '24', ',', '2025', 'Calibration', 'Item', ':', 'Axle', 'Weighing', 'Scale', 'Capacity', ':', '15', '000', 'kgf', 'Measurement', 'Range', ':', '0', 'kgf', 'to', '15', '000', 'kgf', 'Resolution', ':', '5', '0', 'kgf', 'Make', '/', 'Model', ':', 'Intercomp', ';', 'Serial', 'No', '.', ':', '1122YL23003', 'Customer', ':', 'SAVVICE', 'CORPORATION', '3rd', 'Floor', 'BMWC', 'Bldg', '.', ',', 'Cagayan', 'Valley', 'Road', 'Sta', '.', 'Rita', ',', 'Guiginto', ',', 'Bulacan', 'MEASUREMENT', 'RESULTS', ':', 'UNCERTAINTY', 'OF', 'MEASUREMENT', ':', 'The', 'uncertainty', 'stated', 'is', 'the', 'expanded', 'uncertainty', 'obtained', 'by', 'multiplying', 'the', 'standard', 'uncertainty', 'by', 'the', 'coverage', 'factor', 'k', '=', '2', '.', 'It', 'has', 'been', 'determined', 'in', 'accordance', 'with', 'the', '“', 'JCGM', '100', ':', '2008', 'Evaluation', 'o

In [3]:
import pdfplumber

def extract_text(path):
    with pdfplumber.open(path) as pdf:
        return "\n".join(page.extract_text() or "" for page in pdf.pages)

pdf_path = r"C:\Users\ADMIN\Documents\GitHub\Digital-Calibration-Certificate-for-National-Metrology-Laboratory-of-the-Philippines\valid pdf\tensilemachine.pdf"
raw_text = extract_text(pdf_path)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [4]:
raw_text

'CALIBRATION CERTIFICATE\nNo. 02-2019-FORC-0028\nDate of Calibration : March 13, 2019\nCalibration Item : Tensile Testing Machine\nCapacity : 2000 kgf\nMeasurement Range : 0 kgf to 1000 kgf\nResolution : 0 . 1 :\nMake / Model : Gotech/AI-7000-M (Load Cell); Fairbanks/GT1110 (Indicator)\nSerial No. : TC150801739 (Load Cell); t1232131 (Indicator)\nCustomer : SYCWIN COATING & WIRES, INC.\n93 Joy St., Grace Village, Balingasa, Quezon City\nMEASUREMENT RESULTS:\nRelative Relative\nApplied Relative\nIndicated Force Expanded Repeatability\nForce Accuracy Error Class\nUncertainty Error\nkN kN kN % % %\n0.0 0.00 0.00 0.00 0.00 0.00 -\n100.0 100.00 100.76 0.37 -0.76 0.58 1\n200.0 200.00 200.19 0.18 -0.10 0.13 1\n300.0 300.00 300.12 0.27 -0.04 0.35 1\n400.0 400.00 400.12 0.16 -0.04 0.02 1\n500.0 500.00 499.95 0.19 0.02 0.17 1\nUNCERTAINTY OF MEASUREMENT:\nThe uncertainty stated is the expanded uncertainty obtained by multiplying the standard uncertainty\nby the coverage factor k = 2. It has been 

In [5]:
def extract_calibration_info(tokens):
    # Initialize variables to store extracted information
    info = {
        'certificate_number': '',
        'calibration_date': '',
        'item': '',
        'capacity': '',
        'measurement_range': '',
        'resolution': '',
        'make_model': '',
        'serial_number': '',
        'customer': '',
        'customer_address': ''
    }
    
    # List of valid month names for date detection
    months = ['January', 'February', 'March', 'April', 'May', 'June', 
              'July', 'August', 'September', 'October', 'November', 'December']
    
    # Extract certificate number
    try:
        cert_idx = -1
        for i, token in enumerate(tokens):
            if token == 'No' and i + 1 < len(tokens) and tokens[i + 1] == '.':
                cert_idx = i + 2
                break
        
        if cert_idx != -1:
            # Build certificate number by joining tokens until 'Date' or other delimiter
            cert_parts = []
            current_idx = cert_idx
            while current_idx < len(tokens) and tokens[current_idx] not in ['Date', 'Calibration']:
                cert_parts.append(tokens[current_idx])
                current_idx += 1
            
            # Join the parts to form complete certificate number
            info['certificate_number'] = ''.join(cert_parts)
    except Exception:
        pass
    
    # Extract calibration date by looking for a valid month
    try:
        # Find the month in the tokens
        month_idx = -1
        for i, token in enumerate(tokens):
            if token in months:
                month_idx = i
                break
        
        if month_idx != -1:
            # Extract the complete date (month, day, year)
            # Typically the format is: Month Day, Year
            date_parts = []
            current_idx = month_idx
            # Add month
            date_parts.append(tokens[current_idx])
            current_idx += 1
            
            # Add day
            if current_idx < len(tokens) and tokens[current_idx].isdigit():
                date_parts.append(tokens[current_idx])
                current_idx += 1
            
            # Add comma if present
            if current_idx < len(tokens) and tokens[current_idx] == ',':
                date_parts.append(tokens[current_idx])
                current_idx += 1
            
            # Add year
            if current_idx < len(tokens) and tokens[current_idx].isdigit():
                date_parts.append(tokens[current_idx])
            
            # Join the parts to form complete date
            info['calibration_date'] = ' '.join(date_parts)
    except Exception:
        pass
    
    # Extract item
    try:
        item_idx = tokens.index('Item') + 2  # Skip 'Item', ':'
        capacity_idx = tokens.index('Capacity')
        info['item'] = ' '.join(tokens[item_idx:capacity_idx])
    except ValueError:
        pass
    
    # Extract capacity
    try:
        capacity_idx = tokens.index('Capacity') + 2  # Skip 'Capacity', ':'
        measurement_idx = tokens.index('Measurement')
        # Join the numbers and unit
        info['capacity'] = ' '.join(tokens[capacity_idx:measurement_idx])
    except ValueError:
        pass
    
    # Extract measurement range
    try:
        range_idx = tokens.index('Range') + 2  # Skip 'Range', ':'
        resolution_idx = tokens.index('Resolution')
        info['measurement_range'] = ' '.join(tokens[range_idx:resolution_idx])
    except ValueError:
        pass
    
    # Extract resolution
    try:
        resolution_idx = tokens.index('Resolution') + 2  # Skip 'Resolution', ':'
        make_idx = tokens.index('Make')
        # Join the resolution value
        info['resolution'] = ''.join(tokens[resolution_idx:make_idx])
    except ValueError:
        pass
    
    # Extract make/model
    try:
        make_idx = tokens.index('Model') + 2  # Skip 'Model', ':'
        serial_idx = tokens.index('Serial')
        info['make_model'] = ' '.join(tokens[make_idx:serial_idx])
    except ValueError:
        pass
    
    # Extract serial number
    try:
        serial_idx = tokens.index('Serial') + 4  # Skip 'Serial', 'No', '.', ':'
        customer_idx = tokens.index('Customer')
        info['serial_number'] = tokens[serial_idx]
    except ValueError:
        pass
    
    # Extract customer and address with improved logic
    try:
        customer_idx = tokens.index('Customer') + 2  # Skip 'Customer', ':'
        
        # First, identify the customer name
        customer_name_tokens = []
        address_start_idx = -1
        
        # Look for address indicators
        for i in range(customer_idx, len(tokens)):
            if tokens[i] in ['Floor', 'St', 'Street', 'Ave', 'Avenue', 'Road', 'Bldg', '3rd']:
                address_start_idx = i - 1  # Start from the word before the identifier
                break
            if i > customer_idx:  # Add tokens to customer name until we find address start
                customer_name_tokens.append(tokens[i])
        
        # If no address indicators found, use a reasonable cutoff
        if address_start_idx == -1:
            address_start_idx = min(customer_idx + len(customer_name_tokens), len(tokens) - 1)
        
        # Set customer name
        info['customer'] = ' '.join(customer_name_tokens)
        
        # Now extract the address
        address_end_idx = -1
        for i in range(address_start_idx, len(tokens)):
            if tokens[i] in ['MEASUREMENT', 'RESULTS', 'UNCERTAINTY']:
                address_end_idx = i
                break
        
        if address_end_idx == -1:
            address_end_idx = len(tokens) - 1  # If no end marker found, go to end
            
        # Extract address, starting from the address indicator
        info['customer_address'] = ' '.join(tokens[address_start_idx:address_end_idx])
    except ValueError:
        pass
    
    # Clean up data - remove any unwanted characters or fix formatting issues
    for key in info:
        # Remove extra spaces
        info[key] = ' '.join(info[key].split())
        # Fix common issues
        info[key] = info[key].replace(' ,', ',').replace(' .', '.').replace(' ;', ';')
    
    return info


info = extract_calibration_info(toks)

# Print the extracted information
for key, value in info.items():
    print(f"{key}: {value}")

NameError: name 'toks' is not defined

In [ ]:
# # define token‐based start/end markers (all in lowercase)
# FIELD_PATTERNS_TOK = {
#     "cert_no": (["calibration","certificate","no","."], ["date","of","calibration",":"]),
#     "calibration_date": (["date","of","calibration",":"],  ["calibration","item",":"]),
#     "calibration_item": (["calibration","item",":"], ["capacity",":"]),
#     "capacity": (["capacity",":"], ["measurement","range",":"]),
#     "measurement_range": (["measurement","range",":"], ["resolution",":"]),
#     "resolution": (["resolution",":"], ["make","/","model",":"]),
#     "make_model": (["make","/","model",":"], ["serial","no",".",":"]),
#     "serial_no": (["serial","no",".",":"], ["customer",":"]),
#     "customer": (["customer",":"], ["measurement","results",":"]),
# }

# # for keys where you want spaces between tokens
# SPACE_KEYS = {"calibration_item", "capacity", "measurement_range"}  # ← customize

# def find_sublist(lst, sub):
#     """Return first index where sub appears in lst, else -1."""
#     n, m = len(lst), len(sub)
#     for i in range(n - m + 1):
#         if lst[i:i+m] == sub:
#             return i
#     return -1

# tokens_norm = [t.lower() for t in toks]
# data = {}

# for key, (start_seq, end_seq) in FIELD_PATTERNS_TOK.items():
#     i = find_sublist(tokens_norm, start_seq)
#     if i < 0:
#         data[key] = None
#         continue

#     j_rel = find_sublist(tokens_norm[i+len(start_seq):], end_seq)
#     if j_rel < 0:
#         data[key] = None
#         continue

#     j = i + len(start_seq) + j_rel
#     slice_tokens = toks[i+len(start_seq):j]

#     if key in SPACE_KEYS:
#         # join with spaces
#         data[key] = " ".join(slice_tokens).strip()
#     else:
#         # join without spaces
#         data[key] = "".join(slice_tokens).strip()

# data


{'cert_no': '02-2019-FORC-0028',
 'calibration_date': 'March13,2019',
 'calibration_item': 'Tensile Testing Machine',
 'capacity': '2000 kgf',
 'measurement_range': '0 kgf to 1000 kgf',
 'resolution': '0.1:',
 'make_model': 'Gotech/AI-7000-M(LoadCell);Fairbanks/GT1110(Indicator)',
 'serial_no': 'TC150801739(LoadCell);t1232131(Indicator)',
 'customer': 'SYCWINCOATING&WIRES,INC.93JoySt.,GraceVillage,Balingasa,QuezonCity'}

In [ ]:
from datetime import datetime
import re

import xml.etree.ElementTree as ET

# 1) your extracted values
software_name          = "DigiCert" #STATIC
software_release       = "1.0" #STATIC
country_code_iso       = "PH" #STATIC
used_lang_code         = "en" #STATIC
mandatory_lang_code    = "en" #STATIC
# take the actual certificate number string, not the token list

# CERTIFICATE NO. (TSR Number)
unique_identifier = data.get("cert_no", "")  


# DATE
raw_date = data.get("calibration_date", "")
# ensure a space between month name and day, then normalize commas and whitespace
step1 = re.sub(r"([A-Za-z]+)(\d)", r"\1 \2", raw_date)
step2 = step1.replace(",", ", ")
norm  = re.sub(r"\s+", " ", step2).strip()

try:
    # parse “FullMonthName DD, YYYY” → “YYYY‑MM‑DD”
    dt = datetime.strptime(norm, "%B %d, %Y")
    begin_performance_date = dt.strftime("%Y-%m-%d")
except ValueError:
    begin_performance_date = ""

end_performance_date   = ""

#PERFORMANCE LOCATION
performance_location   = "LABORATORY" #ONSITE or LABORATORY


# Calibration item
item_name             = data.get("calibration_item", "")
equipment_ref_type    = "" 
equipment_reference   = "" 
equipment_class_id    = ""

#Standard item
subitem_name    = ""
subitem_model         = ""
ident_issuer          = ""
ident_value           = ""
ident_name            = ""

lab_code              = "FORC" #STATIC
lab_contact_name      = "National Metrology Laboratory - Industrial Technology Development Institute" #STATIC
lab_city              = "Taguig" #STATIC
lab_country_code      = "PH" #STATIC
lab_post_code         = "1633" #STATIC
lab_street            = "General Santos Ave" #STATIC

resp1_name            = ""
resp1_role            = ""
resp2_name            = ""
resp2_role            = ""
resp3_name            = ""
resp3_role            = ""

customer_name         = ""
customer_city         = ""
customer_country_code = "PH" #STATIC
customer_post_code    = ""
customer_street       = ""
customer_street_no    = ""

comment               = ""

# 2) parse + register namespaces
template = r"valid xml\empty.xml"
output   = r"valid xml\filled.xml"

ns = {
    "dcc": "https://ptb.de/dcc",
    "si":  "https://ptb.de/si"
}
for prefix, uri in ns.items():
    ET.register_namespace(prefix, uri)

tree = ET.parse(template)
root = tree.getroot()

def set_text(elem, txt, lang=None):
    if txt is None:
        return
    elem.text = txt
    if lang:
        elem.set("lang", lang)

# 3) fill in
set_text(root.find(".//dcc:software/dcc:name/dcc:content", ns), software_name)
set_text(root.find(".//dcc:software/dcc:release",           ns), software_release)
set_text(root.find(".//dcc:coreData/dcc:countryCodeISO3166_1", ns), country_code_iso)
set_text(root.find(".//dcc:coreData/dcc:usedLangCodeISO639_1",  ns), used_lang_code)
set_text(root.find(".//dcc:coreData/dcc:mandatoryLangCodeISO639_1", ns), mandatory_lang_code)
set_text(root.find(".//dcc:coreData/dcc:uniqueIdentifier",       ns), unique_identifier)
set_text(root.find(".//dcc:coreData/dcc:beginPerformanceDate",   ns), begin_performance_date)
set_text(root.find(".//dcc:coreData/dcc:endPerformanceDate",     ns), end_performance_date)
set_text(root.find(".//dcc:coreData/dcc:performanceLocation",    ns), performance_location)

# items
set_text(root.find(".//dcc:items/dcc:name/dcc:content", ns), item_name, lang=used_lang_code)
ec = root.find(".//dcc:items/dcc:equipmentClass", ns)
if ec is not None:
    ec.set("refType", equipment_ref_type)
    set_text(ec.find("dcc:reference", ns), equipment_reference)
    set_text(ec.find("dcc:classID",   ns), equipment_class_id)

# sub‐item + identification
si_el = root.find(".//dcc:item", ns)
if si_el is not None:
    set_text(si_el.find("dcc:name/dcc:content", ns), subitem_name, lang=used_lang_code)
    set_text(si_el.find("dcc:model", ns), subitem_model)
    ident = si_el.find("dcc:identifications/dcc:identification", ns)
    if ident is not None:
        set_text(ident.find("dcc:issuer",               ns), ident_issuer)
        set_text(ident.find("dcc:value",                ns), ident_value)
        set_text(ident.find("dcc:name/dcc:content",     ns), ident_name, lang=used_lang_code)

# laboratory
set_text(root.find(".//dcc:calibrationLaboratory/dcc:calibrationLaboratoryCode", ns), lab_code)
set_text(root.find(".//dcc:calibrationLaboratory/dcc:contact/dcc:name/dcc:content", ns), lab_contact_name, lang=used_lang_code)
set_text(root.find(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:city",        ns), lab_city)
set_text(root.find(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:countryCode",   ns), lab_country_code)
set_text(root.find(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:postCode",     ns), lab_post_code)
set_text(root.find(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:street",       ns), lab_street)

# respPersons
resp_nodes = root.findall(".//dcc:respPersons/dcc:respPerson", ns)
for idx, (name, role) in enumerate([
    (resp1_name, resp1_role),
    (resp2_name, resp2_role),
    (resp3_name, resp3_role),
]):
    if idx < len(resp_nodes):
        rp = resp_nodes[idx]
        set_text(rp.find("dcc:person/dcc:name/dcc:content", ns), name, lang=used_lang_code)
        set_text(rp.find("dcc:role", ns), role)

# customer
cust = root.find(".//dcc:customer", ns)
if cust is not None:
    set_text(cust.find("dcc:name/dcc:content", ns), customer_name, lang=used_lang_code)
    loc = cust.find("dcc:location", ns)
    set_text(loc.find("dcc:city", ns),        customer_city)
    set_text(loc.find("dcc:countryCode", ns), customer_country_code)
    set_text(loc.find("dcc:postCode", ns),    customer_post_code)
    set_text(loc.find("dcc:street", ns),      customer_street)
    set_text(loc.find("dcc:streetNo", ns),    customer_street_no)

# comment
set_text(root.find(".//dcc:comment", ns), comment)

# 4) write
tree.write(output, encoding="utf-8", xml_declaration=True)

In [ ]:
# import pdfplumber
# import re

# def tokenize_pdf(pdf_path):
#     import re
#     import pdfplumber

#     TOK_RE = r"\w+|[^\w\s]"
#     tokens = []      # for text-only tokens
#     table_tokens = [] # for table tokens

#     with pdfplumber.open(pdf_path) as pdf:
#         for page in pdf.pages:
#             # 1) extract & tokenize tables
#             tables = page.find_tables()
#             for table in tables:
#                 for row in table.extract():             # row is a list of cell‑strings
#                     for cell in row:
#                         if cell:
#                             table_tokens += re.findall(TOK_RE, cell, re.UNICODE)

#             # 2) extract text from the page and tokenize
#             # Create a list of table bounding boxes to check against
#             table_bboxes = [table.bbox for table in tables]
            
#             # Extract all words from the page with their bounding boxes
#             words = page.extract_words()
#             for word in words:
#                 # Check if this word's bbox overlaps with any table bbox
#                 word_bbox = (word['x0'], word['top'], word['x1'], word['bottom'])
#                 in_table = False
#                 for table_bbox in table_bboxes:
#                     # Simple overlap check: not (word is completely left, right, above or below table)
#                     if not (word_bbox[2] < table_bbox[0] or  # word is left of table
#                             word_bbox[0] > table_bbox[2] or  # word is right of table
#                             word_bbox[3] < table_bbox[1] or  # word is above table
#                             word_bbox[1] > table_bbox[3]):   # word is below table
#                         in_table = True
#                         break
                
#                 # Only add tokens from words not in tables
#                 if not in_table:
#                     tokens += re.findall(TOK_RE, word['text'], re.UNICODE)

#     return tokens, table_tokens

# if __name__ == "__main__":
#     pdf_path = r"C:\Work\DOST-ITDI\Digital-Calibration-Certificate-for-National-Metrology-Laboratory-of-the-Philippines\valid pdf\1.pdf"
#     toks, table_toks = tokenize_pdf(pdf_path)
#     # show first 200 tokens for brevity
#     print("Text tokens:")
#     print(toks)
#     print(f"... total text tokens: {len(toks)}")
    
#     print("\nTable tokens:")
#     print(table_toks)
#     print(f"... total table tokens: {len(table_toks)}")

Text tokens:
['CALIBRATION', 'CERTIFICATE', 'No', '.', '02', '-', '2025', '-', 'FORC', '-', '0043', 'Date', 'of', 'Calibration', ':', 'February', '24', ',', '2025', 'Calibration', 'Item', ':', 'Axle', 'Weighing', 'Scale', 'Capacity', ':', '15', '000', 'kgf', 'Measurement', 'Range', ':', '0', 'kgf', 'to', '15', '000', 'kgf', 'Resolution', ':', '5', '0', 'kgf', 'Make', '/', 'Model', ':', 'Intercomp', ';', 'Serial', 'No', '.', ':', '1122YL23002', 'Customer', ':', 'SAVVICE', 'CORPORATION', '3rd', 'Floor', 'BMWC', 'Bldg', '.', ',', 'Cagayan', 'Valley', 'Road', 'Sta', '.', 'Rita', ',', 'Guiginto', ',', 'Bulacan', 'MEASUREMENT', 'RESULTS', ':', 'UNCERTAINTY', 'OF', 'MEASUREMENT', ':', 'The', 'uncertainty', 'stated', 'is', 'the', 'expanded', 'uncertainty', 'obtained', 'by', 'multiplying', 'the', 'standard', 'uncertainty', 'by', 'the', 'coverage', 'factor', 'k', '=', '2', '.', 'It', 'has', 'been', 'determined', 'in', 'accordance', 'with', 'the', '“', 'JCGM', '100', ':', '2008', 'Evaluation', 'o

In [ ]:
# # define token‐based start/end markers (all in lowercase)
# FIELD_PATTERNS_TOK = {
#     "cert_no": (["calibration","certificate","no","."], ["date","of","calibration",":"]),
#     "calibration_date": (["date","of","calibration",":"],  ["calibration","item",":"]),
#     "calibration_item": (["calibration","item",":"], ["capacity",":"]),
#     "capacity": (["capacity",":"], ["measurement","range",":"]),
#     "measurement_range": (["measurement","range",":"], ["resolution",":"]),
#     "resolution": (["resolution",":"], ["make","/","model",":"]),
#     "make_model": (["make","/","model",":"], ["serial","no",".",":"]),
#     "serial_no": (["serial","no",".",":"], ["customer",":"])
# }

# # for keys where you want spaces between tokens
# SPACE_KEYS = {"calibration_item", "capacity"}  # ← customize

# def find_sublist(lst, sub):
#     """Return first index where sub appears in lst, else -1."""
#     n, m = len(lst), len(sub)
#     for i in range(n - m + 1):
#         if lst[i:i+m] == sub:
#             return i
#     return -1

# tokens_norm = [t.lower() for t in toks]
# data = {}

# for key, (start_seq, end_seq) in FIELD_PATTERNS_TOK.items():
#     i = find_sublist(tokens_norm, start_seq)
#     if i < 0:
#         data[key] = None
#         continue

#     j_rel = find_sublist(tokens_norm[i+len(start_seq):], end_seq)
#     if j_rel < 0:
#         data[key] = None
#         continue

#     j = i + len(start_seq) + j_rel
#     slice_tokens = toks[i+len(start_seq):j]

#     if key in SPACE_KEYS:
#         # join with spaces
#         data[key] = " ".join(slice_tokens).strip()
#     else:
#         # join without spaces
#         data[key] = "".join(slice_tokens).strip()

# data


{'cert_no': '02-2025-FORC-0043',
 'calibration_date': 'February24,2025',
 'calibration_item': 'Axle Weighing Scale',
 'capacity': '15 000 kgf',
 'measurement_range': '0kgfto15000kgf',
 'resolution': '50kgf',
 'make_model': 'Intercomp;',
 'serial_no': '1122YL23002'}

In [ ]:
# from datetime import datetime
# import re

# import xml.etree.ElementTree as ET

# # 1) your extracted values
# software_name          = "DigiCert" #STATIC
# software_release       = "1.0" #STATIC
# country_code_iso       = "PH" #STATIC
# used_lang_code         = "en" #STATIC
# mandatory_lang_code    = "en" #STATIC
# # take the actual certificate number string, not the token list

# # CERTIFICATE NO. (TSR Number)
# unique_identifier = data.get("cert_no", "")  


# # DATE
# raw_date = data.get("calibration_date", "")
# # ensure a space between month name and day, then normalize commas and whitespace
# step1 = re.sub(r"([A-Za-z]+)(\d)", r"\1 \2", raw_date)
# step2 = step1.replace(",", ", ")
# norm  = re.sub(r"\s+", " ", step2).strip()

# try:
#     # parse “FullMonthName DD, YYYY” → “YYYY‑MM‑DD”
#     dt = datetime.strptime(norm, "%B %d, %Y")
#     begin_performance_date = dt.strftime("%Y-%m-%d")
# except ValueError:
#     begin_performance_date = ""

# end_performance_date   = ""

# #PERFORMANCE LOCATION
# performance_location   = "LABORATORY" #ONSITE or LABORATORY


# # Calibration item
# item_name             = data.get("calibration_item", "")
# equipment_ref_type    = "" 
# equipment_reference   = "" 
# equipment_class_id    = ""

# #Standard item
# subitem_name    = ""
# subitem_model         = ""
# ident_issuer          = ""
# ident_value           = ""
# ident_name            = ""

# lab_code              = "FORC" #STATIC
# lab_contact_name      = "National Metrology Laboratory - Industrial Technology Development Institute" #STATIC
# lab_city              = "Taguig" #STATIC
# lab_country_code      = "PH" #STATIC
# lab_post_code         = "1633" #STATIC
# lab_street            = "General Santos Ave" #STATIC

# resp1_name            = ""
# resp1_role            = ""
# resp2_name            = ""
# resp2_role            = ""
# resp3_name            = ""
# resp3_role            = ""

# customer_name         = ""
# customer_city         = ""
# customer_country_code = "PH" #STATIC
# customer_post_code    = ""
# customer_street       = ""
# customer_street_no    = ""

# comment               = ""

# # 2) parse + register namespaces
# template = r"valid xml\empty.xml"
# output   = r"valid xml\filled.xml"

# ns = {
#     "dcc": "https://ptb.de/dcc",
#     "si":  "https://ptb.de/si"
# }
# for prefix, uri in ns.items():
#     ET.register_namespace(prefix, uri)

# tree = ET.parse(template)
# root = tree.getroot()

# def set_text(elem, txt, lang=None):
#     if txt is None:
#         return
#     elem.text = txt
#     if lang:
#         elem.set("lang", lang)

# # 3) fill in
# set_text(root.find(".//dcc:software/dcc:name/dcc:content", ns), software_name)
# set_text(root.find(".//dcc:software/dcc:release",           ns), software_release)
# set_text(root.find(".//dcc:coreData/dcc:countryCodeISO3166_1", ns), country_code_iso)
# set_text(root.find(".//dcc:coreData/dcc:usedLangCodeISO639_1",  ns), used_lang_code)
# set_text(root.find(".//dcc:coreData/dcc:mandatoryLangCodeISO639_1", ns), mandatory_lang_code)
# set_text(root.find(".//dcc:coreData/dcc:uniqueIdentifier",       ns), unique_identifier)
# set_text(root.find(".//dcc:coreData/dcc:beginPerformanceDate",   ns), begin_performance_date)
# set_text(root.find(".//dcc:coreData/dcc:endPerformanceDate",     ns), end_performance_date)
# set_text(root.find(".//dcc:coreData/dcc:performanceLocation",    ns), performance_location)

# # items
# set_text(root.find(".//dcc:items/dcc:name/dcc:content", ns), item_name, lang=used_lang_code)
# ec = root.find(".//dcc:items/dcc:equipmentClass", ns)
# if ec is not None:
#     ec.set("refType", equipment_ref_type)
#     set_text(ec.find("dcc:reference", ns), equipment_reference)
#     set_text(ec.find("dcc:classID",   ns), equipment_class_id)

# # sub‐item + identification
# si_el = root.find(".//dcc:item", ns)
# if si_el is not None:
#     set_text(si_el.find("dcc:name/dcc:content", ns), subitem_name, lang=used_lang_code)
#     set_text(si_el.find("dcc:model", ns), subitem_model)
#     ident = si_el.find("dcc:identifications/dcc:identification", ns)
#     if ident is not None:
#         set_text(ident.find("dcc:issuer",               ns), ident_issuer)
#         set_text(ident.find("dcc:value",                ns), ident_value)
#         set_text(ident.find("dcc:name/dcc:content",     ns), ident_name, lang=used_lang_code)

# # laboratory
# set_text(root.find(".//dcc:calibrationLaboratory/dcc:calibrationLaboratoryCode", ns), lab_code)
# set_text(root.find(".//dcc:calibrationLaboratory/dcc:contact/dcc:name/dcc:content", ns), lab_contact_name, lang=used_lang_code)
# set_text(root.find(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:city",        ns), lab_city)
# set_text(root.find(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:countryCode",   ns), lab_country_code)
# set_text(root.find(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:postCode",     ns), lab_post_code)
# set_text(root.find(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:street",       ns), lab_street)

# # respPersons
# resp_nodes = root.findall(".//dcc:respPersons/dcc:respPerson", ns)
# for idx, (name, role) in enumerate([
#     (resp1_name, resp1_role),
#     (resp2_name, resp2_role),
#     (resp3_name, resp3_role),
# ]):
#     if idx < len(resp_nodes):
#         rp = resp_nodes[idx]
#         set_text(rp.find("dcc:person/dcc:name/dcc:content", ns), name, lang=used_lang_code)
#         set_text(rp.find("dcc:role", ns), role)

# # customer
# cust = root.find(".//dcc:customer", ns)
# if cust is not None:
#     set_text(cust.find("dcc:name/dcc:content", ns), customer_name, lang=used_lang_code)
#     loc = cust.find("dcc:location", ns)
#     set_text(loc.find("dcc:city", ns),        customer_city)
#     set_text(loc.find("dcc:countryCode", ns), customer_country_code)
#     set_text(loc.find("dcc:postCode", ns),    customer_post_code)
#     set_text(loc.find("dcc:street", ns),      customer_street)
#     set_text(loc.find("dcc:streetNo", ns),    customer_street_no)

# # comment
# set_text(root.find(".//dcc:comment", ns), comment)

# # 4) write
# tree.write(output, encoding="utf-8", xml_declaration=True)

In [ ]:
# import xml.etree.ElementTree as ET

# # 1) Declare one variable per tag (initially empty)
# software_name            = "Digital Calibration Certificate"  # <dcc:name><dcc:content>
# software_release         = "1.0"  # <dcc:release>
# country_code_iso         = "PHL"  # <dcc:countryCodeISO3166_1>
# used_lang_code           = "en"  # <dcc:usedLangCodeISO639_1>
# mandatory_lang_code      = "en"  # <dcc:mandatoryLangCodeISO639_1>
# unique_identifier        = "01-2025-FORC-0013"  # <dcc:uniqueIdentifier>
# begin_performance_date   = "2025-02-24"  # <dcc:beginPerformanceDate>
# end_performance_date     = ""  # <dcc:endPerformanceDate>
# performance_location     = "Marikina, Eastern Manila District, NCR"  # <dcc:performanceLocation>

# item_name                = "Axle Weighing Scale"  # <dcc:items>/<dcc:name>/<dcc:content>
# equipment_ref_type       = "instrument"  # equipmentClass refType=
# equipment_reference      = "APOLLO / AW565"  # <dcc:equipmentClass>/<dcc:reference>
# equipment_class_id       = "weighing_scale"  # <dcc:equipmentClass>/<dcc:classID>

# subitem_name             = "Axle Weighing Scale"  # <dcc:item>/<dcc:name>/<dcc:content>
# subitem_model            = "APOLLO / AW565"  # <dcc:item>/<dcc:model>
# ident_issuer             = "Manufacturer"  # <dcc:identification>/<dcc:issuer>
# ident_value              = "A386856"  # <dcc:identification>/<dcc:value>
# ident_name               = "Serial Number"  # <dcc:identification>/<dcc:name>/<dcc:content>

# lab_code                 = "NMD-ITDI"  # <dcc:calibrationLaboratoryCode>
# lab_contact_name         = "National Metrology Laboratory"  # <dcc:contact>/<dcc:name>/<dcc:content>
# lab_city                 = "Manila"  # <dcc:location>/<dcc:city>
# lab_country_code         = "PHL"  # <dcc:location>/<dcc:countryCode>
# lab_post_code            = ""  # <dcc:location>/<dcc:postCode>
# lab_street               = ""  # <dcc:location>/<dcc:street>

# resp1_name               = "AHDRIAN CAMILO C. GERNALE"  # first <dcc:respPerson>/<dcc:person>/<dcc:name>/<dcc:content>
# resp1_role               = "Science Research Specialist II"  # first <dcc:respPerson>/<dcc:role>
# resp2_name               = "RADLEY F. MANALO"  # second respPerson...
# resp2_role               = "Senior Science Research Specialist"
# resp3_name               = "MARYNESS I. SALAZAR, PhD"  # third respPerson...
# resp3_role               = "Head, Pressure and Force Standards Section"

# customer_name            = ""  # <dcc:customer>/<dcc:name>/<dcc:content>
# customer_city            = "Marikina"  # <dcc:customer>/<dcc:location>/<dcc:city>
# customer_country_code    = "PHL"  # <…>/countryCode
# customer_post_code       = ""  # <…>/postCode
# customer_street          = "No. 4 Malaya Street Malanday"  # <…>/street
# customer_street_no       = "4"  # <…>/streetNo

# comment                  = "The axle weighing scale was calibrated in static compression mode."  # <dcc:comment>

# # 2) Parse template and inject
# template_path = r"valid xml\empty.xml"
# output_path   = r"valid xml\filled.xml"

# ns = {
#     "dcc": "https://ptb.de/dcc",
#     "si":  "https://ptb.de/si"
# }

# # register your prefixes so ET will reuse them instead of ns0, ns1…
# for prefix, uri in ns.items():
#     ET.register_namespace(prefix, uri)

# tree = ET.parse(template_path)
# root = tree.getroot()

# def set_text(path, value):
#     el = root.find(path, ns)
#     if el is not None:
#         el.text = value
        

# set_text(".//dcc:software/dcc:name/dcc:content", software_name)
# set_text(".//dcc:software/dcc:release", software_release)
# set_text(".//dcc:coreData/dcc:countryCodeISO3166_1", country_code_iso)
# set_text(".//dcc:coreData/dcc:usedLangCodeISO639_1", used_lang_code)
# set_text(".//dcc:coreData/dcc:mandatoryLangCodeISO639_1", mandatory_lang_code)
# set_text(".//dcc:coreData/dcc:uniqueIdentifier", unique_identifier)
# set_text(".//dcc:coreData/dcc:beginPerformanceDate", begin_performance_date)
# set_text(".//dcc:coreData/dcc:endPerformanceDate", end_performance_date)
# set_text(".//dcc:coreData/dcc:performanceLocation", performance_location)

# set_text(".//dcc:items/dcc:name/dcc:content", item_name)
# set_text(".//dcc:items/dcc:equipmentClass", equipment_ref_type)        # via attribute
# set_text(".//dcc:items/dcc:equipmentClass/dcc:reference", equipment_reference)
# set_text(".//dcc:items/dcc:equipmentClass/dcc:classID", equipment_class_id)

# set_text(".//dcc:item/dcc:name/dcc:content", subitem_name)
# set_text(".//dcc:item/dcc:model", subitem_model)
# set_text(".//dcc:identification/dcc:issuer", ident_issuer)
# set_text(".//dcc:identification/dcc:value", ident_value)
# set_text(".//dcc:identification/dcc:name/dcc:content", ident_name)

# set_text(".//dcc:calibrationLaboratory/dcc:calibrationLaboratoryCode", lab_code)
# set_text(".//dcc:calibrationLaboratory/dcc:contact/dcc:name/dcc:content", lab_contact_name)
# set_text(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:city", lab_city)
# set_text(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:countryCode", lab_country_code)
# set_text(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:postCode", lab_post_code)
# set_text(".//dcc:calibrationLaboratory/dcc:contact/dcc:location/dcc:street", lab_street)

# set_text(".//dcc:respPersons/dcc:respPerson[1]/dcc:person/dcc:name/dcc:content", resp1_name)
# set_text(".//dcc:respPersons/dcc:respPerson[1]/dcc:role", resp1_role)
# set_text(".//dcc:respPersons/dcc:respPerson[2]/dcc:person/dcc:name/dcc:content", resp2_name)
# set_text(".//dcc:respPersons/dcc:respPerson[2]/dcc:role", resp2_role)
# set_text(".//dcc:respPersons/dcc:respPerson[3]/dcc:person/dcc:name/dcc:content", resp3_name)
# set_text(".//dcc:respPersons/dcc:respPerson[3]/dcc:role", resp3_role)

# set_text(".//dcc:customer/dcc:name/dcc:content", customer_name)
# set_text(".//dcc:customer/dcc:location/dcc:city", customer_city)
# set_text(".//dcc:customer/dcc:location/dcc:countryCode", customer_country_code)
# set_text(".//dcc:customer/dcc:location/dcc:postCode", customer_post_code)
# set_text(".//dcc:customer/dcc:location/dcc:street", customer_street)
# set_text(".//dcc:customer/dcc:location/dcc:streetNo", customer_street_no)

# set_text(".//dcc:comment", comment)

# # 3) Write out your filled XML
# tree.write(output_path, encoding="utf-8", xml_declaration=True)

In [ ]:
# # Install necessary libraries:
# # pip install PyPDF2

# import PyPDF2
# import re
# import xml.etree.ElementTree as ET
# from xml.dom import minidom # For pretty printing XML
# import datetime # For date parsing

# # --- Helper Functions ---

# def extract_text_from_pdf(pdf_path):
#     """Extracts text content from all pages of a PDF file."""
#     text = ""
#     try:
#         # It's crucial to ensure the PDF path is correct and accessible
#         with open(pdf_path, 'rb') as pdf_file:
#             reader = PyPDF2.PdfReader(pdf_file)
#             num_pages = len(reader.pages)
#             print(f"Reading {num_pages} pages from {pdf_path}...")
#             for page_num in range(num_pages):
#                 page = reader.pages[page_num]
#                 # Extract text using PyPDF2
#                 page_text = page.extract_text()
#                 if page_text:
#                     text += page_text + "\n\n" # Add double newline between pages for separation
#                 else:
#                     print(f"Warning: Could not extract text from page {page_num + 1}.")
#         print("Finished extracting text.")
#         # Basic cleaning: replace multiple spaces/newlines which can affect regex
#         text = re.sub(r'\s{2,}', ' ', text) # Replace multiple spaces with one
#         text = re.sub(r'\n+', '\n', text)   # Replace multiple newlines with one
#         return text
#     except FileNotFoundError:
#         print(f"Error: PDF file not found at '{pdf_path}'")
#         return None
#     except Exception as e:
#         print(f"An error occurred during PDF reading: {e}")
#         return None

# def find_pattern(text, pattern, label="pattern", group=1, default=None, flags=re.IGNORECASE | re.MULTILINE):
#     """Uses regex to find a pattern and returns the specified capturing group or default."""
#     if not text:
#         return default
#     try:
#         match = re.search(pattern, text, flags)
#         if match:
#             try:
#                 # Return the specified capturing group, stripping whitespace
#                 return match.group(group).strip()
#             except IndexError:
#                 # Only print warning if label is provided
#                 if label:
#                     print(f"Warning: Regex pattern for '{label}' matched, but group {group} not found.")
#                 return default
#             except Exception as e:
#                  # Only print warning if label is provided
#                  if label:
#                      print(f"Warning: Error accessing group {group} for '{label}': {e}")
#                  return default
#         else:
#             # Only print warning if label is provided and pattern wasn't found
#             # if label:
#             #     print(f"Debug: Could not find {label} using pattern: {pattern}") # Less verbose
#             return default
#     except Exception as e:
#         # Only print warning if label is provided
#         if label:
#             print(f"An error occurred during regex search for '{label}': {e}")
#         return default

# def parse_date(date_str, label="date"):
#     """Parses common date formats and returns YYYY-MM-DD."""
#     if not date_str:
#         return None
#     try:
#         # Example: "February 24, 2025"
#         dt_object = datetime.datetime.strptime(date_str.strip(), "%B %d, %Y")
#         return dt_object.strftime("%Y-%m-%d")
#     except ValueError:
#         print(f"Warning: Could not parse {label} string '{date_str}' into YYYY-MM-DD format.")
#         return None

# def extract_table_data(text):
#     """
#     Extracts data from the measurement results table.
#     NOTE: This is highly dependent on the text extraction format and spacing.
#           This is FRAGILE and might need a more robust PDF table parsing library.
#     """
#     table_data = {
#         "Applied Force": [],
#         "Indicated Force": [],
#         "Deviation": [],
#         "Relative Expanded Uncertainty": [],
#         "Relative Accuracy Error": [],
#     }
#     if not text:
#         return table_data

#     # Try to find the start of the table data rows (after the header)
#     header_match = re.search(r"Applied\s+Force\s+Indicated\s+Force.*?Error\s*kgf\s+kgf\s+kgf\s+%\s+%", text, re.IGNORECASE | re.DOTALL)

#     if not header_match:
#         print("Warning: Could not find measurement results table header.")
#         return table_data # Return empty dict if header not found

#     # Extract text *after* the header
#     table_content = text[header_match.end():]

#     # Find the end of the table (e.g., start of "UNCERTAINTY OF MEASUREMENT")
#     end_match = re.search(r"UNCERTAINTY OF MEASUREMENT", table_content, re.IGNORECASE)
#     if end_match:
#         table_content = table_content[:end_match.start()]

#     # Split into lines and process rows containing numerical data
#     lines = table_content.strip().split('\n')
#     # Regex to capture the 5 columns, allowing for spaces within numbers (like "2 900") and negative signs
#     row_pattern = re.compile(r"^\s*([\d\.\-]+)\s+([\d\s\.\-]+)\s+([\d\.\-]+)\s+([\d\.]+)\s+([\d\.]+)\s*$")

#     found_rows = 0
#     for line in lines:
#         line = line.strip()
#         # Clean up potential issues like "2 900" -> "2900" before matching
#         cleaned_line = re.sub(r'(\d)\s+(\d)', r'\1\2', line)
#         # Handle potential negative sign separated by space like "- 50" -> "-50"
#         cleaned_line = re.sub(r'-\s+(\d)', r'-\1', cleaned_line)

#         match = row_pattern.match(cleaned_line)
#         if match:
#             try:
#                 # Further clean spaces from matched groups if necessary (though sub above should handle most)
#                 table_data["Applied Force"].append(match.group(1).replace(" ", ""))
#                 table_data["Indicated Force"].append(match.group(2).replace(" ", ""))
#                 table_data["Deviation"].append(match.group(3).replace(" ", ""))
#                 table_data["Relative Expanded Uncertainty"].append(match.group(4).replace(" ", ""))
#                 table_data["Relative Accuracy Error"].append(match.group(5).replace(" ", ""))
#                 found_rows += 1
#             except IndexError:
#                 print(f"Warning: Could not parse all columns from table row: {line}")
#         # else:
#             # print(f"Debug: Line did not match table row pattern: '{line}' (cleaned: '{cleaned_line}')") # Uncomment for debugging

#     if found_rows == 0:
#          print("Warning: Failed to extract any numerical data rows from the table.")
#          return {} # Return empty dict if no rows parsed
#     else:
#          print(f"Successfully extracted {found_rows} rows of table data.")
#          # print("Table Data:", table_data) # Uncomment for debugging

#     return table_data

# # --- Main Data Extraction Function ---

# def extract_calibration_data(text):
#     """
#     Extracts specific data points from the text using regex.
#     Only stores data explicitly found in the text.
#     """
#     if not text:
#         return None

#     data = {}
#     print("Attempting to extract data fields...")

#     # --- Administrative Data ---
#     # Use find_pattern which returns None if not found
#     data['uniqueIdentifier'] = find_pattern(text, r"No\.\s+([0-9\-A-Z]+)", "Unique Identifier")
#     raw_date = find_pattern(text, r"Date of Calibration\s*:\s*([A-Za-z]+\s+\d{1,2},\s+\d{4})", "Calibration Date String")
#     data['calibrationDate'] = parse_date(raw_date, "Calibration Date")
#     # Performance dates often same as calibration date, assign if calibration date found
#     if data['calibrationDate']:
#         data['beginPerformanceDate'] = data['calibrationDate']
#         data['endPerformanceDate'] = data['calibrationDate']
#     # Performance location is not explicitly stated in the sample PDF
#     # data['performanceLocation'] = find_pattern(text, r"Location:\s*(.+)", "Performance Location") # Example if pattern existed

#     # Country/Language codes inferred from context (document language, addresses)
#     # Only include if reasonably certain from context, otherwise omit.
#     # Checking for NCR in address implies PH.
#     if find_pattern(text, r"NCR", label=None): # Check if NCR is mentioned
#          data['countryCodeISO3166_1'] = "PH" # Inferred
#          print("Info: Inferred country code 'PH' based on context (NCR mention).")
#     data['usedLangCodeISO639_1'] = "en" # Inferred from document language
#     data['mandatoryLangCodeISO639_1'] = "en" # Inferred from document language
#     print("Info: Inferred language codes 'en' based on document language.")


#     # --- Item Information ---
#     data['item_name'] = find_pattern(text, r"Calibration Item\s*:\s*(.+?)\s*(?:Capacity|Measurement Range|Make/Model)", "Item Name")
#     data['item_model'] = find_pattern(text, r"Make/Model\s*:\s*([^;\n]+)", "Item Model") # Stop at semicolon or newline
#     data['item_serial_number'] = find_pattern(text, r"Serial No\.\s*:\s*(\S+)", "Item Serial Number")

#     # --- Standard Used (Equipment Class) ---
#     standard_section_match = re.search(r"STANDARD USED:(.*?)(CALIBRATION PROCEDURE:|ENVIRONMENTAL CONDITIONS:)", text, re.IGNORECASE | re.DOTALL)
#     standard_text = standard_section_match.group(1) if standard_section_match else ""
#     if standard_text:
#         data['standard_ref_sn'] = find_pattern(standard_text, r"SN\s+(\S+)", "Standard SN")
#         raw_standard_model = find_pattern(standard_text, r"Shimadzu/\s*(UH-?F1000kNX)", "Standard Model") # Handle optional hyphen
#         if raw_standard_model:
#              # Clean up standard model if found
#              data['standard_model'] = raw_standard_model.replace('-', '')
#     else:
#         print("Warning: Could not find 'STANDARD USED' section.")


#     # --- Calibration Laboratory ---
#     # Extract lab code from identifier if possible and found
#     if data.get('uniqueIdentifier'):
#         match = re.search(r'-([A-Z]+)-', data['uniqueIdentifier'])
#         if match:
#             data['calibrationLaboratoryCode'] = match.group(1)
#         else:
#              print("Warning: Could not extract laboratory code from unique identifier.")
#     # Lab name: Look for explicit mention or specific context line
#     # The line "For the Chief..." is context, not the actual lab name. Look for NML-ITDI explicitly.
#     data['lab_name'] = find_pattern(text, r"(NML-ITDI)", "Lab Name Explicit")
#     if not data.get('lab_name'):
#          # Fallback: Check if standard traceability mentions it
#          data['lab_name'] = find_pattern(standard_text, r"NMD-ITDI", "Lab Name from Standard Traceability")

#     # Lab location details are not present in the sample PDF text. Do not add.


#     # --- Responsible Persons ---
#     # Use find_pattern which returns None if not found
#     data['respPerson1_name'] = find_pattern(text, r"(AHDRIAN CAMILO C\. GERNALE)\s+Science Research Specialist II", "Resp Person 1 Name")
#     if data.get('respPerson1_name'): data['respPerson1_role'] = "Science Research Specialist II"

#     data['respPerson2_name'] = find_pattern(text, r"(RADLEY F\. MANALO)\s+Senior Science Research Specialist", "Resp Person 2 Name")
#     if data.get('respPerson2_name'): data['respPerson2_role'] = "Senior Science Research Specialist"

#     # Person 3 name has comma, need to be careful with regex if matching role on same line
#     # Match name first, then look for role nearby if needed
#     data['respPerson3_name'] = find_pattern(text, r"(MARYNESS I\. SALAZAR, PhD)", "Resp Person 3 Name")
#     if data.get('respPerson3_name'):
#         # Check if role follows immediately or on next line (adjust pattern if needed)
#         role3_match = re.search(re.escape(data['respPerson3_name']) + r"\s+(Head, Pressure and Force Standards Section)", text)
#         if role3_match:
#             data['respPerson3_role'] = role3_match.group(1).strip()
#         else:
#             # Try looking for the role on its own line if structure varies
#             data['respPerson3_role'] = find_pattern(text, r"Head, Pressure and Force Standards Section", "Resp Person 3 Role")


#     # --- Customer ---
#     data['customer_name'] = find_pattern(text, r"Customer\s*:\s*(.+?)\s*(?:No\.|MEASUREMENT RESULTS)", "Customer Name")
#     # Extract address lines more carefully
#     customer_block_match = re.search(r"Customer\s*:\s*.+?\n\s*(.+?)\n\s*(.+?)(?:\n|$)", text, re.IGNORECASE)
#     if customer_block_match:
#         addr_line1 = customer_block_match.group(1).strip()
#         addr_line2 = customer_block_match.group(2).strip()

#         # Try to parse Street No, Street from line 1
#         street_no_match = re.match(r"(No\.\s*\d+)\s*(.*)", addr_line1)
#         if street_no_match:
#             data['customer_streetNo'] = street_no_match.group(1)
#             data['customer_street'] = street_no_match.group(2)
#         else:
#             # If no "No." prefix, assume the whole line is the street
#             data['customer_street'] = addr_line1

#         # Try to parse City from line 2 (usually before comma or end of line)
#         city_match = re.match(r"([^,]+)", addr_line2)
#         if city_match:
#             data['customer_city'] = city_match.group(1).strip()

#         # Post code is not present in the address lines in the PDF text. Do not add.
#         # data['customer_postCode'] = find_pattern(addr_line2, r"(\d{4})", "Customer Postcode") # Example if it existed

#     # Customer country code inferred from context (NCR mention)
#     if data.get('countryCodeISO3166_1'): # Use lab's inferred country code if available
#          data['customer_countryCode'] = data['countryCodeISO3166_1']


#     # --- Measurement Results Table ---
#     data['measurement_table'] = extract_table_data(text)

#     print("Finished extracting data fields.")
#     # Remove keys with None values before returning for cleaner XML generation check
#     cleaned_data = {k: v for k, v in data.items() if v is not None and v != ""}
#     # Special check for table data - keep if not empty dict
#     if 'measurement_table' in data and data['measurement_table']:
#          cleaned_data['measurement_table'] = data['measurement_table']
#     elif 'measurement_table' in cleaned_data: # remove if it ended up empty
#          del cleaned_data['measurement_table']


#     print("Final data keys to be used for XML:", list(cleaned_data.keys()))
#     return cleaned_data


# # --- XML Generation Function ---

# def create_dcc_xml(data, output_xml_path):
#     """
#     Creates a DCC XML file based *only* on the extracted data provided.
#     Checks for key existence before adding elements.
#     """
#     if not data:
#         print("Error: No valid data extracted to create XML.")
#         return

#     print(f"Creating XML file at {output_xml_path}...")

#     # --- Define Namespaces ---
#     NS_DCC = "https://ptb.de/dcc"
#     NS_SI = "https://ptb.de/si"
#     NS_XSI = "http://www.w3.org/2001/XMLSchema-instance"
#     ET.register_namespace('dcc', NS_DCC)
#     ET.register_namespace('si', NS_SI)
#     ET.register_namespace('xsi', NS_XSI)

#     # --- Root Element ---
#     root = ET.Element(f"{{{NS_DCC}}}digitalCalibrationCertificate",
#                       attrib={
#                           f"{{{NS_XSI}}}schemaLocation": f"{NS_DCC} https://ptb.de/dcc/v3.2.1/dcc.xsd",
#                           "schemaVersion": "3.2.1" # Schema version is required by schemaLocation
#                       })

#     # --- Helper to create namespaced elements with text ---
#     def create_element(parent, ns, tag, text=None, attrib=None):
#         # Only create element if parent exists and (text is valid or it's meant to be a container)
#         if parent is not None and (text is not None or attrib is not None):
#              element = ET.SubElement(parent, f"{{{ns}}}{tag}", attrib=attrib if attrib else {})
#              if text is not None:
#                  element.text = str(text)
#              return element
#         return None # Return None if parent is None or no text/attributes provided

#     # --- Helper to create dcc:content elements ---
#     def create_dcc_content(parent, ns, tag, content_text, lang="en"):
#          if content_text and parent is not None: # Check both content and parent
#              name_elem = ET.SubElement(parent, f"{{{ns}}}{tag}")
#              content_elem = ET.SubElement(name_elem, f"{{{ns}}}content", attrib={"lang": lang})
#              content_elem.text = str(content_text)
#              return name_elem
#          return None

#     # --- Administrative Data ---
#     admin_data = create_element(root, NS_DCC, "administrativeData") # Create main container

#     # Software Info (Optional - Example shows it, but not from PDF)
#     # If you want to include this, uncomment the following lines.
#     # dcc_software = create_element(admin_data, NS_DCC, "dccSoftware")
#     # if dcc_software is not None:
#     #     software = create_element(dcc_software, NS_DCC, "software")
#     #     if software is not None:
#     #         create_dcc_content(software, NS_DCC, "name", "Python DCC Generator") # Example name
#     #         create_element(software, NS_DCC, "release", "1.1") # Example release

#     # Core Data - Create container only if there's core data to add
#     core_data_items = {
#         'countryCodeISO3166_1': data.get('countryCodeISO3166_1'),
#         'usedLangCodeISO639_1': data.get('usedLangCodeISO639_1'),
#         'mandatoryLangCodeISO639_1': data.get('mandatoryLangCodeISO639_1'),
#         'uniqueIdentifier': data.get('uniqueIdentifier'),
#         'beginPerformanceDate': data.get('beginPerformanceDate'),
#         'endPerformanceDate': data.get('endPerformanceDate'),
#         'performanceLocation': data.get('performanceLocation') # Will likely be None
#     }
#     # Filter out None values
#     valid_core_data = {k: v for k, v in core_data_items.items() if v is not None}
#     if valid_core_data:
#         core_data = create_element(admin_data, NS_DCC, "coreData")
#         for key, value in valid_core_data.items():
#             create_element(core_data, NS_DCC, key, value) # Use dict key as tag name

#     # Items - Create container only if item name or details exist
#     if data.get('item_name') or data.get('standard_ref_sn') or data.get('standard_model') or data.get('item_model') or data.get('item_serial_number'):
#         items = create_element(admin_data, NS_DCC, "items")
#         create_dcc_content(items, NS_DCC, "name", data.get('item_name'))

#         # Equipment Class (Standard Used) - Create only if standard data exists
#         if data.get('standard_ref_sn') or data.get('standard_model'):
#             equip_class = create_element(items, NS_DCC, "equipmentClass", attrib={"refType": "Force Measuring Instrument"})
#             if equip_class is not None:
#                 create_element(equip_class, NS_DCC, "reference", data.get('standard_ref_sn'))
#                 create_element(equip_class, NS_DCC, "classID", data.get('standard_model'))

#         # Item Details - Create container only if model or serial number exists
#         if data.get('item_model') or data.get('item_serial_number'):
#             item = create_element(items, NS_DCC, "item")
#             if item is not None:
#                 create_dcc_content(item, NS_DCC, "name", data.get('item_name')) # Repeat item name inside item
#                 create_element(item, NS_DCC, "model", data.get('item_model'))

#                 # Identifications (Serial Number) - Create only if serial number exists
#                 if data.get('item_serial_number'):
#                     identifications = create_element(item, NS_DCC, "identifications")
#                     if identifications is not None:
#                         identification = create_element(identifications, NS_DCC, "identification")
#                         if identification is not None:
#                             create_element(identification, NS_DCC, "issuer", "customer") # Assumption
#                             create_element(identification, NS_DCC, "value", data.get('item_serial_number'))
#                             create_dcc_content(identification, NS_DCC, "name", data.get('item_model')) # Use model as name

#     # Calibration Laboratory - Create only if code or contact info exists
#     if data.get('calibrationLaboratoryCode') or data.get('lab_name'):
#         calib_lab = create_element(admin_data, NS_DCC, "calibrationLaboratory")
#         if calib_lab is not None:
#             create_element(calib_lab, NS_DCC, "calibrationLaboratoryCode", data.get('calibrationLaboratoryCode'))
#             # Contact - Create only if name or location details exist (name is likely only one found)
#             if data.get('lab_name'): # Add other conditions if lab location fields were added back
#                 contact = create_element(calib_lab, NS_DCC, "contact")
#                 if contact is not None:
#                     create_dcc_content(contact, NS_DCC, "name", data.get('lab_name'))
#                     # Location - Create only if location details exist (unlikely for lab in this PDF)
#                     # lab_location_data = {k: data.get(f"lab_{k}") for k in ["city", "countryCode", "postCode", "street"] if data.get(f"lab_{k}")}
#                     # if lab_location_data:
#                     #    location = create_element(contact, NS_DCC, "location")
#                     #    for key, value in lab_location_data.items():
#                     #        create_element(location, NS_DCC, key, value)

#     # Responsible Persons - Create container only if at least one person found
#     resp_persons_data = []
#     for i in range(1, 4):
#         name_key = f'respPerson{i}_name'
#         role_key = f'respPerson{i}_role'
#         if data.get(name_key) and data.get(role_key):
#             resp_persons_data.append({'name': data[name_key], 'role': data[role_key]})

#     if resp_persons_data:
#         resp_persons = create_element(admin_data, NS_DCC, "respPersons")
#         if resp_persons is not None:
#             for person_data in resp_persons_data:
#                 resp_person = create_element(resp_persons, NS_DCC, "respPerson")
#                 if resp_person is not None:
#                     person = create_element(resp_person, NS_DCC, "person")
#                     if person is not None:
#                         create_dcc_content(person, NS_DCC, "name", person_data['name'])
#                     create_element(resp_person, NS_DCC, "role", person_data['role'])

#     # Customer - Create container only if name or location details exist
#     cust_location_data = {k: data.get(f"customer_{k}") for k in ["city", "countryCode", "postCode", "street", "streetNo"] if data.get(f"customer_{k}")}
#     if data.get('customer_name') or cust_location_data:
#          customer = create_element(admin_data, NS_DCC, "customer")
#          if customer is not None:
#              create_dcc_content(customer, NS_DCC, "name", data.get('customer_name'))
#              # Location - Create only if location details exist
#              if cust_location_data:
#                  cust_location = create_element(customer, NS_DCC, "location")
#                  if cust_location is not None:
#                     # Map dict keys back to XML tags (handle streetNo case)
#                     tag_map = {"city": "city", "countryCode": "countryCode", "postCode": "postCode", "street": "street", "streetNo": "streetNo"}
#                     for key, value in cust_location_data.items():
#                         create_element(cust_location, NS_DCC, tag_map[key], value)


#     # --- Measurement Results ---
#     table = data.get('measurement_table')
#     # Create container only if item name exists OR table data exists
#     if data.get('item_name') or (table and any(table.values())):
#         meas_results = create_element(root, NS_DCC, "measurementResults")
#         if meas_results is not None:
#             meas_result = create_element(meas_results, NS_DCC, "measurementResult")
#             if meas_result is not None:
#                 create_dcc_content(meas_result, NS_DCC, "name", data.get('item_name'))

#                 # Create results/result containers only if table data exists
#                 if table and any(table.values()):
#                     results = create_element(meas_result, NS_DCC, "results")
#                     if results is not None:
#                         result = create_element(results, NS_DCC, "result")
#                         if result is not None:
#                             create_dcc_content(result, NS_DCC, "name", "Calibration Test Results") # Static name for this section

#                             dcc_data = create_element(result, NS_DCC, "data")
#                             if dcc_data is not None:
#                                 # Map PDF column names to XML list names and SI types
#                                 column_map = {
#                                     "Applied Force": ("Applied Force", "constant", "kgf"),
#                                     "Indicated Force": ("Indicated Force", "real", "kgf"),
#                                     "Deviation": ("Deviation (Indicated Force - Applied Force)", "real", "kgf"),
#                                     "Relative Expanded Uncertainty": ("Relative Expanded Uncertainty", "real", "%"),
#                                     "Relative Accuracy Error": ("Relative Accuracy Error", "real", "%"),
#                                 }

#                                 for pdf_col, (xml_name, si_type, unit) in column_map.items():
#                                     # Check if column exists in extracted table data and has values
#                                     if pdf_col in table and table[pdf_col]:
#                                         dcc_list = create_element(dcc_data, NS_DCC, "list")
#                                         if dcc_list is not None:
#                                             create_dcc_content(dcc_list, NS_DCC, "name", xml_name)
#                                             for value in table[pdf_col]:
#                                                 # Ensure value is not None or empty before creating quantity
#                                                 if value is not None and value != "":
#                                                     quantity = create_element(dcc_list, NS_DCC, "quantity")
#                                                     if quantity is not None:
#                                                         si_elem = create_element(quantity, NS_SI, si_type)
#                                                         if si_elem is not None:
#                                                             create_element(si_elem, NS_SI, "value", value)
#                                                             create_element(si_elem, NS_SI, "unit", unit)


#     # --- Convert ElementTree to string with pretty printing ---
#     try:
#         # Use encoding='unicode' for minidom processing
#         rough_string = ET.tostring(root, encoding='unicode', method='xml')
#         if not rough_string:
#              print("Error: XML generation resulted in an empty string.")
#              return

#         # Add the XML declaration which ET.tostring omits
#         xml_declaration = '<?xml version="1.0" encoding="UTF-8"?>\n'

#         # Use minidom for pretty printing
#         reparsed = minidom.parseString(rough_string)
#         pretty_xml_as_string = reparsed.toprettyxml(indent="  ")

#         # Remove the default declaration added by minidom and add ours
#         pretty_xml_as_string = xml_declaration + '\n'.join(pretty_xml_as_string.split('\n')[1:])

#         # Write to file
#         with open(output_xml_path, "w", encoding='utf-8') as f:
#             f.write(pretty_xml_as_string)
#         print(f"Successfully created pretty XML file: {output_xml_path}")

#     except Exception as e:
#         print(f"Error during XML generation/writing: {e}")
#         # Fallback: Write the non-prettified version if minidom fails
#         try:
#             # Re-generate rough string if needed, ensure it's unicode for writing
#             rough_string_fallback = ET.tostring(root, encoding='unicode', method='xml')
#             xml_declaration_fallback = '<?xml version="1.0" encoding="UTF-8"?>\n'
#             with open(output_xml_path, "w", encoding='utf-8') as f:
#                  f.write(xml_declaration_fallback + rough_string_fallback) # Write unicode string
#             print(f"Successfully created basic XML file (no pretty print): {output_xml_path}")
#         except Exception as write_e:
#             print(f"Error writing basic XML file: {write_e}")


# # --- Main Execution ---
# if __name__ == "__main__":
#     # IMPORTANT: Replace with the actual path to your PDF file if needed
#     pdf_input_path = "TISSI-sample-certificate.pdf" # Assumes file is in the same directory

#     # IMPORTANT: Replace with the desired path for the output XML file
#     xml_output_path = "output_dcc_strict.xml" # Changed output filename

#     # 1. Extract text from PDF
#     extracted_text = extract_text_from_pdf(pdf_input_path)

#     if extracted_text:
#         # 2. Extract specific data using patterns (strict mode)
#         calibration_data = extract_calibration_data(extracted_text)

#         # 3. Create the XML file based *only* on extracted data
#         if calibration_data: # Check if dict is not empty
#             create_dcc_xml(calibration_data, xml_output_path)
#         else:
#             print("No data was successfully extracted from the PDF. XML file not generated.")
#     else:
#         print("Failed to extract text from the PDF. Cannot proceed.")


Reading 2 pages from TISSI-sample-certificate.pdf...
Finished extracting text.
Attempting to extract data fields...
Info: Inferred language codes 'en' based on document language.
Finished extracting data fields.
Final data keys to be used for XML: ['uniqueIdentifier', 'usedLangCodeISO639_1', 'mandatoryLangCodeISO639_1', 'item_name', 'item_serial_number', 'calibrationLaboratoryCode', 'respPerson1_name', 'respPerson1_role', 'respPerson3_name', 'respPerson3_role', 'customer_street', 'customer_city', 'measurement_table']
Creating XML file at output_dcc_strict.xml...
Successfully created pretty XML file: output_dcc_strict.xml


In [ ]:
# import xml.etree.ElementTree as ET
# import re
# import fitz  # Example using PyMuPDF
# # Or: import pdfplumber # Example using pdfplumber
# from datetime import datetime

# # --- 1. Default Values & Configuration ---
# # These might come from a config file or be hardcoded if consistent
# DEFAULT_COUNTRY_CODE = "PH"
# DEFAULT_LANG_CODE = "en"
# DCC_SOFTWARE_NAME = "YourPDFtoDCC_Converter" # Or specific software used
# DCC_SOFTWARE_RELEASE = "v1.0" # Version of your script/tool
# SCHEMA_VERSION = "3.2.1" # Match the schema you are targeting
# DCC_NAMESPACE = "https://ptb.de/dcc"
# SI_NAMESPACE = "https://ptb.de/si"
# XSI_NAMESPACE = "http://www.w3.org/2001/XMLSchema-instance"
# SCHEMA_LOCATION = "https://ptb.de/dcc https://ptb.de/dcc/v3.2.1/dcc.xsd"

# # Register namespaces for cleaner output (especially with lxml)
# ET.register_namespace('dcc', DCC_NAMESPACE)
# ET.register_namespace('si', SI_NAMESPACE)
# ET.register_namespace('xsi', XSI_NAMESPACE)

# # --- 2. Helper Functions ---

# def extract_text_from_pdf(pdf_path):
#     """Extracts text content from all pages of a PDF."""
#     doc = fitz.open(pdf_path)
#     full_text = ""
#     for page_num in range(len(doc)):
#         page = doc.load_page(page_num)
#         full_text += page.get_text("text")
#     doc.close()
#     return full_text

# def find_value_after_keyword(text, keyword, stop_keywords=None):
#     """Simple regex helper to find value after a keyword until newline or stop keyword."""
#     # Make keyword regex safe and flexible with optional colon/space
#     keyword_pattern = re.escape(keyword).replace('\\:', ':?') + r'\s*[:]?\s*(.*)'
#     match = re.search(keyword_pattern, text, re.IGNORECASE | re.MULTILINE)
#     if match:
#         value_line = match.group(1).strip()
#         # Optional: Stop extraction if another keyword appears on the same line
#         if stop_keywords:
#             for stop_kw in stop_keywords:
#                 stop_match = re.search(re.escape(stop_kw), value_line, re.IGNORECASE)
#                 if stop_match:
#                     value_line = value_line[:stop_match.start()].strip()
#                     break
#         return value_line
#     return None

# def format_date(date_str):
#     """Attempts to parse date and format as YYYY-MM-DD."""
#     if not date_str:
#         return None
#     try:
#         # Add more formats if needed
#         date_obj = datetime.strptime(date_str.strip(), '%B %d, %Y')
#         return date_obj.strftime('%Y-%m-%d')
#     except ValueError:
#         print(f"Warning: Could not parse date '{date_str}'")
#         return date_str # Return original if parsing fails

# def extract_table_data(pdf_path, page_num, table_settings=None):
#     """Extracts table data using pdfplumber (conceptual)."""
#     # This requires pdfplumber library: pip install pdfplumber
#     # You'll need to tune table extraction settings heavily based on PDF
#     print(f"Attempting table extraction on page {page_num}...")
#     # try:
#     #     with pdfplumber.open(pdf_path) as pdf:
#     #         page = pdf.pages[page_num - 1] # pdfplumber is 0-indexed
#     #         # Example: Find table automatically (often needs tuning)
#     #         # tables = page.extract_tables(table_settings)
#     #         # Or define bounding box: table = page.crop((x0, top, x1, bottom)).extract_table()
#     #         # Simplified: Assuming one main table found per page relevant section
#     #         tables = page.extract_tables() # This needs refinement
#     #         if tables:
#     #              print(f"Found {len(tables)} tables on page {page_num}.")
#     #              # Select the correct table based on headers or position
#     #              # For now, assume the first relevant table
#     #              # Clean data: remove None rows/cells, strip whitespace, handle line breaks within cells
#     #              cleaned_table = []
#     #              for row in tables[0]: # Assuming first table is the target
#     #                  cleaned_row = [str(cell).strip().replace('\n', ' ') if cell is not None else '' for cell in row]
#     #                  if any(cleaned_row): # Skip empty rows
#     #                      cleaned_table.append(cleaned_row)
#     #              return cleaned_table
#     # except Exception as e:
#     #      print(f"Error extracting table on page {page_num}: {e}")
#     return None # Placeholder

# # --- 3. XML Element Creation Helpers ---
# # These helpers make creating the nested XML structure cleaner

# def create_dcc_element(tag, text=None, attrib=None, lang=None):
#     """Creates an element in the DCC namespace."""
#     if attrib is None:
#         attrib = {}
#     element = ET.Element(f"{{{DCC_NAMESPACE}}}{tag}", attrib)
#     if text:
#         element.text = str(text)
#     if lang:
#         element.set(f"{{{ET.XML_NS}}}lang", lang) # Use xml:lang
#     return element

# def create_si_element(tag, text=None, attrib=None):
#      """Creates an element in the SI namespace."""
#      if attrib is None:
#          attrib = {}
#      element = ET.Element(f"{{{SI_NAMESPACE}}}{tag}", attrib)
#      if text:
#          element.text = str(text)
#      return element

# def create_dcc_text_element(parent, tag, content, lang=DEFAULT_LANG_CODE):
#     """Creates standard <dcc:tag><dcc:content> structure."""
#     elem = create_dcc_element(tag)
#     content_elem = create_dcc_element("content", text=content, lang=lang)
#     elem.append(content_elem)
#     parent.append(elem)
#     return elem

# def create_si_quantity(parent_tag, value, unit):
#     """Creates a simple si:real or si:constant quantity element."""
#     # Determine if it's constant (like applied force) or real (measured)
#     # Simple heuristic: if value contains only digits/decimal point/minus, treat as real?
#     # Or based on parent_tag context. Here, assume 'real' for measured, 'constant' for applied.
#     # Needs refinement based on actual schema usage.
#     # This example uses si:real for simplicity for most measured values.
#     # Applied Force uses si:constant as per the example XML.
#     # Deviation could also be si:real

#     if parent_tag == "Applied Force":
#          q_type = "constant"
#     else:
#          q_type = "real" # Default assumption

#     quantity_elem = create_dcc_element("quantity")
#     si_elem = create_si_element(q_type)
#     si_value = create_si_element("value", text=str(value))
#     si_unit = create_si_element("unit", text=str(unit))
#     si_elem.append(si_value)
#     si_elem.append(si_unit)
#     quantity_elem.append(si_elem)
#     return quantity_elem


# # --- 4. Main Extraction and XML Building Logic ---

# def create_dcc_from_pdf(pdf_path, output_xml_path):
#     """Main function to parse PDF and generate DCC XML."""

#     # --- Extract Data ---
#     full_text = extract_text_from_pdf(pdf_path)
#     # print("Extracted Text:\n", full_text) # For debugging

#     # Extract specific fields using keywords and regex
#     # Page 1 Data
#     cert_no = find_value_after_keyword(full_text, "No.")
#     cal_date_str = find_value_after_keyword(full_text, "Date of Calibration")
#     cal_item = find_value_after_keyword(full_text, "Calibration Item")
#     capacity = find_value_after_keyword(full_text, "Capacity", stop_keywords=["kgf"]) # Simple stop example
#     measurement_range = find_value_after_keyword(full_text, "Measurement Range", stop_keywords=["kgf"])
#     resolution = find_value_after_keyword(full_text, "Resolution", stop_keywords=["kgf"])
#     make_model = find_value_after_keyword(full_text, "Make/Model")
#     serial_no = find_value_after_keyword(full_text, "Serial No.")
#     customer_name = find_value_after_keyword(full_text, "Customer")
#     # Customer address requires more complex multi-line parsing logic
#     customer_address_line1 = find_value_after_keyword(full_text, customer_name) # Assuming address follows name
#     customer_address_line2 = find_value_after_keyword(full_text, customer_address_line1) # Highly simplistic

#     # Page 2 Data (Example - needs robust finding logic)
#     # Standard Used Table Data (Requires proper table extraction)
#     standard_name = "Force Measuring Instrument" # Hardcoded from PDF text/context
#     standard_make_model = "Shimadzu/ UH-F1000kNX"
#     standard_serial = "1251056K0094" # Extracted via keyword near standard name
#     standard_cert_no = "11-2020-FORC-0116"

#     # Personnel
#     person1_name = "AHDRIAN CAMILO C. GERNALE" # Extract based on position or signature block
#     person1_role = "Science Research Specialist II"
#     person2_name = "RADLEY F. MANALO"
#     person2_role = "Senior Science Research Specialist"
#     # Add logic for Dr. Salazar if needed based on schema requirements

#     # Environmental Conditions
#     temp_str = find_value_after_keyword(full_text, "Ambient Temperature")
#     humidity_str = find_value_after_keyword(full_text, "Relative Humidity")

#     # Measurement Results Table Data (CRITICAL - Requires proper table extraction)
#     # This needs a dedicated table extraction call, e.g.:
#     # measurement_table = extract_table_data(pdf_path, page_num=1, table_settings={...})
#     # Fallback/Placeholder - Use data from your example XML if table parsing fails/not implemented
#     measurement_table = [
#          ["Applied Force", "Indicated Force", "Deviation (Indicated Force - Applied Force)", "Relative Expanded Uncertainty", "Relative Accuracy Error"],
#          ["kgf", "kgf", "kgf", "%", "%"], # Units row (may need special handling)
#          ["0.00", "0.00", "0", "0.00", "0.00"],
#          ["3000", "2900", "-100", "1.02", "3.45"], # Handle potential thousands separators if present
#          ["6000", "5900", "-100", "0.54", "1.69"],
#          ["9000", "8950", "-50", "0.40", "0.56"], # Handle minus sign and value merging
#          ["12000", "12000", "0", "0.34", "0.00"],
#          ["15000", "15000", "0", "0.31", "0.00"],
#     ]
#     print("Using placeholder measurement table data.")


#     # --- Build XML ---
#     root_attrib = {
#         f"{{{XSI_NAMESPACE}}}schemaLocation": SCHEMA_LOCATION,
#         "schemaVersion": SCHEMA_VERSION
#     }
#     root = ET.Element(f"{{{DCC_NAMESPACE}}}digitalCalibrationCertificate", root_attrib)

#     # -- Administrative Data --
#     admin_data = create_dcc_element("administrativeData")
#     root.append(admin_data)

#     # Software (Using defaults)
#     dcc_software = create_dcc_element("dccSoftware")
#     admin_data.append(dcc_software)
#     software = create_dcc_element("software")
#     dcc_software.append(software)
#     create_dcc_text_element(software, "name", DCC_SOFTWARE_NAME)
#     create_dcc_element("release", text=DCC_SOFTWARE_RELEASE, parent=software) # Typo in example XML fixed (parent added)

#     # Core Data
#     core_data = create_dcc_element("coreData")
#     admin_data.append(core_data)
#     create_dcc_element("countryCodeISO3166_1", text=DEFAULT_COUNTRY_CODE, parent=core_data)
#     create_dcc_element("usedLangCodeISO639_1", text=DEFAULT_LANG_CODE, parent=core_data)
#     create_dcc_element("mandatoryLangCodeISO639_1", text=DEFAULT_LANG_CODE, parent=core_data)
#     create_dcc_element("uniqueIdentifier", text=cert_no, parent=core_data)
#     create_dcc_element("beginPerformanceDate", text=format_date(cal_date_str), parent=core_data)
#     create_dcc_element("endPerformanceDate", text=format_date(cal_date_str), parent=core_data) # Assuming same day
#     create_dcc_element("performanceLocation", text="laboratory", parent=core_data) # From PDF context

#     # Items
#     items = create_dcc_element("items")
#     admin_data.append(items)
#     create_dcc_text_element(items, "name", cal_item, lang=DEFAULT_LANG_CODE) # Name for the list
#     # You might add equipmentClass here if applicable to the whole list

#     item = create_dcc_element("item") # The actual calibrated item
#     items.append(item)
#     create_dcc_text_element(item, "name", cal_item, lang=DEFAULT_LANG_CODE) # Name for the specific item
#     # Add Manufacturer / Model if available separately
#     if make_model:
#          # Simple splitting, might need refinement
#          parts = make_model.split('/', 1)
#          item_manufacturer_name = parts[0].strip() if len(parts) > 0 else make_model
#          item_model_name = parts[1].strip() if len(parts) > 1 else None

#          # Manufacturer (structure depends on exact schema - using contactNotStrictType idea)
#          # manufacturer_elem = create_dcc_element("manufacturer")
#          # create_dcc_text_element(manufacturer_elem, "name", item_manufacturer_name)
#          # item.append(manufacturer_elem) # Add if schema requires

#          if item_model_name:
#              create_dcc_element("model", text=item_model_name, parent=item)

#     # Item Identifications (Serial No.)
#     identifications = create_dcc_element("identifications")
#     item.append(identifications)
#     identification = create_dcc_element("identification")
#     identifications.append(identification)
#     create_dcc_element("issuer", text="manufacturer", parent=identification) # Assuming manufacturer issued SN
#     create_dcc_element("value", text=serial_no, parent=identification)
#     create_dcc_text_element(identification, "name", "Serial Number", lang=DEFAULT_LANG_CODE)

#     # Calibration Laboratory (Partially hardcoded based on context/example)
#     calib_lab = create_dcc_element("calibrationLaboratory")
#     admin_data.append(calib_lab)
#     # Add calibrationLaboratoryCode if known/consistent, e.g., "FORC"
#     create_dcc_element("calibrationLaboratoryCode", text="FORC", parent=calib_lab)
#     contact_lab = create_dcc_element("contact")
#     calib_lab.append(contact_lab)
#     create_dcc_text_element(contact_lab, "name", "NML-ITDI") # From context/example
#     location_lab = create_dcc_element("location")
#     contact_lab.append(location_lab)
#     create_dcc_element("city", text="Taguig", parent=location_lab) # Context/Example
#     create_dcc_element("countryCode", text=DEFAULT_COUNTRY_CODE, parent=location_lab)
#     # Add postCode, street etc. if available and required by schema

#     # Responsible Persons
#     resp_persons = create_dcc_element("respPersons")
#     admin_data.append(resp_persons)
#     # Person 1
#     resp_person1 = create_dcc_element("respPerson")
#     resp_persons.append(resp_person1)
#     person1 = create_dcc_element("person")
#     resp_person1.append(person1)
#     create_dcc_text_element(person1, "name", person1_name, lang=DEFAULT_LANG_CODE)
#     create_dcc_element("role", text=person1_role, parent=resp_person1)
#     # Person 2
#     resp_person2 = create_dcc_element("respPerson")
#     resp_persons.append(resp_person2)
#     person2 = create_dcc_element("person")
#     resp_person2.append(person2)
#     create_dcc_text_element(person2, "name", person2_name, lang=DEFAULT_LANG_CODE)
#     create_dcc_element("role", text=person2_role, parent=resp_person2)
#     # Add Dr. Salazar similarly if needed

#     # Customer
#     customer = create_dcc_element("customer")
#     admin_data.append(customer)
#     create_dcc_text_element(customer, "name", customer_name, lang=DEFAULT_LANG_CODE)
#     location_cust = create_dcc_element("location")
#     customer.append(location_cust)
#     # Parse and add address components (city, street, postCode, countryCode)
#     # Simple example based on limited parsing:
#     create_dcc_element("street", text=customer_address_line1, parent=location_cust) # Needs proper parsing
#     # create_dcc_element("city", text="Marikina", parent=location_cust) # Needs extraction
#     create_dcc_element("countryCode", text=DEFAULT_COUNTRY_CODE, parent=location_cust)


#     # -- Measurement Results --
#     meas_results_list = create_dcc_element("measurementResults")
#     root.append(meas_results_list)

#     # Add usedMethods, measuringEquipments (Standard Used), influenceConditions here
#     # Example: Influence Condition Temp
#     # influence_conditions = create_dcc_element("influenceConditions")
#     # meas_results_list.append(influence_conditions)
#     # temp_cond = create_dcc_element("influenceCondition")
#     # influence_conditions.append(temp_cond)
#     # create_dcc_text_element(temp_cond, "name", "Ambient Temperature")
#     # temp_data = create_dcc_element("data")
#     # temp_cond.append(temp_data)
#     # # Needs parsing of value/uncertainty from temp_str to create si:hybrid or si:real + uncertainty
#     # # Placeholder: create_dcc_element("text", text=temp_str, parent=temp_data)


#     # Main Measurement Result Section (Processing the table)
#     meas_result = create_dcc_element("measurementResult")
#     meas_results_list.append(meas_result)
#     create_dcc_text_element(meas_result, "name", cal_item, lang=DEFAULT_LANG_CODE) # Name for this result set

#     results = create_dcc_element("results")
#     meas_result.append(results)
#     result = create_dcc_element("result") # The overall table result
#     results.append(result)
#     create_dcc_text_element(result, "name", "Calibration Test Results", lang=DEFAULT_LANG_CODE)

#     data = create_dcc_element("data") # Container for the lists (columns)
#     result.append(data)

#     if measurement_table and len(measurement_table) > 2: # Check if table has header, units, and data
#         headers = measurement_table[0]
#         units = measurement_table[1]
#         num_cols = len(headers)
#         num_rows = len(measurement_table) - 2 # Exclude header and unit rows

#         for col_idx in range(num_cols):
#             list_elem = create_dcc_element("list")
#             data.append(list_elem)
#             create_dcc_text_element(list_elem, "name", headers[col_idx], lang=DEFAULT_LANG_CODE)

#             col_unit = units[col_idx]
#             parent_tag_context = headers[col_idx] # Use header name for context

#             for row_idx in range(num_rows):
#                 data_row_idx = row_idx + 2 # Offset by header/unit rows
#                 value_str = measurement_table[data_row_idx][col_idx]
#                 # Clean value (remove commas, handle potential issues)
#                 cleaned_value = value_str.replace(',', '').strip()
#                 try:
#                     # Attempt conversion to float for numeric check, but keep string for XML
#                     float(cleaned_value)
#                     q_elem = create_si_quantity(parent_tag_context, cleaned_value, col_unit)
#                     list_elem.append(q_elem)
#                 except ValueError:
#                     print(f"Warning: Could not convert value '{cleaned_value}' to number in {headers[col_idx]}. Storing as text or skipping.")
#                     # Optionally add as text or handle error


#     # --- Serialize and Save XML ---
#     tree = ET.ElementTree(root)
#     ET.indent(tree, space="  ", level=0) # Pretty print
#     tree.write(output_xml_path, encoding="UTF-8", xml_declaration=True)
#     print(f"DCC XML saved to: {output_xml_path}")


# # --- Run the process ---
# pdf_file = "TISSI-sample-certificate.pdf" # Make sure this path is correct
# output_xml_file = "TISSI-sample-certificate.xml"

# # Make sure the PDF file exists before running
# import os
# if os.path.exists(pdf_file):
#     create_dcc_from_pdf(pdf_file, output_xml_file)
# else:
#     print(f"Error: PDF file not found at {pdf_file}")

In [6]:
import fitz  # PyMuPDF
import sys
import os

# --- Configuration ---
pdf_path = "valid pdf\TISSI-sample-certificate.pdf"
output_pdf_path = "valid pdf\TISSI-sample-certificate_blocks.pdf"
box_color = (1, 0, 0)  # Red (RGB, values 0-1)
text_color = (1, 0, 0) # Red for the number
box_width = 0.5
font_size = 6

# --- Check if PDF exists ---
if not os.path.exists(pdf_path):
    print(f"Error: Input PDF file not found at '{pdf_path}'")
    sys.exit(1)

#
all_blocks_text = []
block_counter = 0

try:
    doc = fitz.open(pdf_path)
    print(f"Opened PDF: '{pdf_path}'. Processing {len(doc)} pages...")

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        
        blocks = page.get_text("blocks")
        blocks.sort(key=lambda b: (b[1], b[0])) # Sort blocks top-to-bottom, left-to-right

        print(f"  Page {page_num + 1}: Found {len(blocks)} blocks.")

        for i, b in enumerate(blocks):
            block_counter += 1
            x0, y0, x1, y1, text_content, _, _ = b

            # Store text content
            clean_text = text_content.strip()
            all_blocks_text.append(clean_text)

            # Draw bounding box rectangle
            rect = fitz.Rect(x0, y0, x1, y1)
            page.draw_rect(rect, color=box_color, width=box_width)

            # Draw block number near the top-left corner
            text_pos = fitz.Point(x0 + 1, y0 + font_size + 1) # Position slightly inside top-left
            page.insert_text(text_pos,
                              f"{block_counter}",
                              fontsize=font_size,
                              color=text_color,
                              rotate=0)

    # Save the modified document
    doc.save(output_pdf_path, garbage=4, deflate=True)
    print(f"\nSuccessfully saved highlighted PDF to: '{output_pdf_path}'")

except Exception as e:
    print(f"\nAn error occurred during PDF processing: {e}")
finally:
    if 'doc' in locals() and doc:
        doc.close()

# --- Interactive Block Text Display ---
if not all_blocks_text:
    print("No text blocks were extracted.")
    sys.exit(1)

print("\n--- Detected Text Blocks (Start of content) ---")
for i, text in enumerate(all_blocks_text):
    snippet = text.replace('\n', ' ').strip()
    print(f"  Block {i + 1}: {snippet[:80]}...") # Print first 80 chars

print("\nEnter a block number to see its full text, or 'q' to quit.")

while True:
    try:
        user_input = input(f"Enter block number (1-{len(all_blocks_text)}) or 'q': ").strip()

        if user_input.lower() == 'q':
            break

        block_index = int(user_input) - 1 # Convert to 0-based index

        if 0 <= block_index < len(all_blocks_text):
            print("-" * 20 + f" Block {block_index + 1} Text " + "-" * 20)
            print(all_blocks_text[block_index])
            print("-" * (42 + len(str(block_index + 1)))) # Match dashes width
        else:
            print(f"Invalid input. Please enter a number between 1 and {len(all_blocks_text)} or 'q'.")

    except ValueError:
        print("Invalid input. Please enter a number or 'q'.")
    except Exception as e:
        print(f"An error occurred: {e}")

print("\nExiting.")

<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:7: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:7: SyntaxWarning: invalid escape sequence '\T'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7896\3157091896.py:6: SyntaxWarning: invalid escape sequence '\T'
  pdf_path = "valid pdf\TISSI-sample-certificate.pdf"
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7896\3157091896.py:7: SyntaxWarning: invalid escape sequence '\T'
  output_pdf_path = "valid pdf\TISSI-sample-certificate_blocks.pdf"


Opened PDF: 'valid pdf\TISSI-sample-certificate.pdf'. Processing 2 pages...
  Page 1: Found 25 blocks.
  Page 2: Found 26 blocks.

An error occurred during PDF processing: code=2: cannot remove file 'valid pdf\TISSI-sample-certificate_blocks.pdf': Permission denied

--- Detected Text Blocks (Start of content) ---
  Block 1: CALIBRATION CERTIFICATE...
  Block 2: No. 01-2025-FORC-0013...
  Block 3: February 24, 2025 Axle Weighing Scale 15 000 kgf 0 kgf to 15 000 kgf 50 kgf APOL...
  Block 4: Date of Calibration : Calibration Item : Capacity : Measurement Range : Resoluti...
  Block 5: Marikina, Eastern Manila District, NCR...
  Block 6: MEASUREMENT RESULTS:...
  Block 7: Relative  Expanded  Uncertainty...
  Block 8: Relative  Accuracy...
  Block 9: Deviation (Indicated Force -...
  Block 10: Applied...
  Block 11: Indicated...
  Block 12: Force...
  Block 13: Force...
  Block 14: Error...
  Block 15: Applied Force)...
  Block 16: kgf kgf kgf % %...
  Block 17: 0.00 0.00 0 0.00 0.00...
  